# Setting

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import matplotlib.font_manager as fm
import platform
plt.rcParams['font.family'] = 'AppleGothic'

# Preprocessing

## 서울시 건설 알림이 정보

In [61]:
construction_df = pd.read_csv('data/raw_data/서울시 건설 알림이 정보.csv', encoding='cp949')

In [62]:
construction_df.head()

,프로젝트 코드,프로젝트 명,사업착수일(계약일),사업기간,프로젝트 종료여부(진행:0 종료:1),사무실주소,프로젝트 주소,위치좌표(위도),위치좌표(경도),자치구 구분,사업금액(억원)
0,3502024051377,2024년 중부수도사업소 관내 포장도로 굴착복구공사(장기1차- 단가계약),202405,2024-05-13~2025-06-30,0,NaN,서울 중구 장충단로 88,37.566779,126.978618,NaN,NaN
1,7552024061894,"2024년 1,2호선 강북구간 궤도시설 보수보강공사",202406,2024-06-18~2025-06-24,0,NaN,서울 중구 정동 5-8,37.565345,126.977198,중구,NaN
2,8502024070466,의회청사 공간재배치 및 내진보강 소방공사 (장기1차),202407,2024-07-04~2025-06-09,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN
3,8502024061277,의회청사 공간재배치 및 내진보강 전기공사 (장기1차),202406,2024-06-12~2025-05-31,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN
4,8502024062180,의회청사 공간재배치 및 내진보강 정보통신공사 (장기1차),202406,2024-06-21~2025-05-31,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN


In [63]:
# 프로젝트 코드 컬럼 삭제 (의미 없음)
construction_df.drop(columns=["프로젝트 코드"], inplace=True)

In [64]:
# 사무실 주소 컬럼 삭제 (의미 없음)
construction_df.drop(columns=["사무실주소"], inplace=True)

In [65]:
# 프로젝트 종료여부(진행:0 종료:1)

# 컬럼 이름 변경
construction_df = construction_df.rename(columns={"프로젝트 종료여부(진행:0 종료:1)": "프로젝트 종료여부"})

# 0 → False, 1 → True 변환
construction_df["프로젝트 종료여부"] = construction_df["프로젝트 종료여부"].map({0: False, 1: True})

In [66]:
# 위도 경도 결측치 제거
construction_df.dropna(subset=["위치좌표(위도)", "위치좌표(경도)"], inplace=True)

In [67]:
# 자취구 컬럼 결측 보완
construction_df["자치구 구분"] = construction_df["자치구 구분"].fillna(
    construction_df["프로젝트 주소"].str.extract(r"\s([^\s]+구)")[0]
)

In [68]:
construction_df.isnull().sum()

프로젝트 명          0
사업착수일(계약일)      0
사업기간            0
프로젝트 종료여부       0
프로젝트 주소        12
위치좌표(위도)        0
위치좌표(경도)        0
자치구 구분         53
사업금액(억원)      537
dtype: int64

In [69]:
# '프로젝트 주소'와 '자치구 구분' 이렇게 했는데도 결측치가 있는 행 제거
construction_df.dropna(subset=["프로젝트 주소", "자치구 구분"], inplace=True)

In [70]:
# 사업착수일 int → datetime
construction_df["사업착수일"] = pd.to_datetime(construction_df["사업착수일(계약일)"].astype(str) + "01", format="%Y%m%d")

# 사업기간 → 시작일, 종료일 분리 및 datetime 변환
construction_df[["사업시작일", "사업종료일"]] = construction_df["사업기간"].str.split("~", expand=True)
construction_df["사업시작일"] = pd.to_datetime(construction_df["사업시작일"], errors="coerce")
construction_df["사업종료일"] = pd.to_datetime(construction_df["사업종료일"], errors="coerce")

# 기존 컬럼 삭제
construction_df.drop(columns=["사업착수일(계약일)"], inplace=True)
construction_df.drop(columns=["사업기간"], inplace=True)

In [71]:
construction_df.isnull().sum()

프로젝트 명         0
프로젝트 종료여부      0
프로젝트 주소        0
위치좌표(위도)       0
위치좌표(경도)       0
자치구 구분         0
사업금액(억원)     506
사업착수일          0
사업시작일          0
사업종료일          0
dtype: int64

In [72]:
construction_df.head()

,프로젝트 명,프로젝트 종료여부,프로젝트 주소,위치좌표(위도),위치좌표(경도),자치구 구분,사업금액(억원),사업착수일,사업시작일,사업종료일
0,2024년 중부수도사업소 관내 포장도로 굴착복구공사(장기1차- 단가계약),False,서울 중구 장충단로 88,37.566779,126.978618,중구,NaN,2024-05-01,2024-05-13,2025-06-30
1,"2024년 1,2호선 강북구간 궤도시설 보수보강공사",False,서울 중구 정동 5-8,37.565345,126.977198,중구,NaN,2024-06-01,2024-06-18,2025-06-24
2,의회청사 공간재배치 및 내진보강 소방공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-07-01,2024-07-04,2025-06-09
3,의회청사 공간재배치 및 내진보강 전기공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-06-01,2024-06-12,2025-05-31
4,의회청사 공간재배치 및 내진보강 정보통신공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-06-01,2024-06-21,2025-05-31


In [73]:
construction_df.to_csv("data/processed/construction_processed.csv", index=False, encoding="utf-8-sig")

## 서울시 건축물 유출지하수 현황

In [119]:
building_underground_water_df = pd.read_csv('data/raw_data/서울시 건축물 유출지하수 현황.csv', encoding='cp949')

In [120]:
building_underground_water_df.head()

,건축물명,건축물 위치,층수(지상_지하),연면적,조사년도,총발생량(톤),일평균발생량(톤/일),일평균이용현황(톤/일)_하천방류,일평균이용현황(톤/일)_도로청소,일평균이용현황(톤/일)_공원용수,일평균이용현황(톤/일)_화장실세척,일평균이용현황(톤/일)_건물용수,미사용_하수도방류(톤/일),일평균이용현황(톤/일)_기타건물용수
0,서초오퓨런스빌딩,서초대로 254(서초동 1553-5),18/8,31215.79,2024,6870.5,37.75,NaN,NaN,NaN,NaN,0.0,37.75,NaN
1,더클래스효성(주),효령로55길 65(서초동 1597-7),/,NaN,2024,2318.7,12.74,NaN,NaN,NaN,NaN,0.0,12.74,NaN
2,웅진타워,반포대로30길 81(서초동),16/8,13852.05,2024,21481.5,118.03,NaN,NaN,NaN,NaN,0.0,118.03,NaN
3,나비호텔레지던스(김영*),서초대로58길 5(서초동 1676-2),11/2,2086.08,2024,358.5,1.97,NaN,NaN,NaN,NaN,0.0,1.97,NaN
4,북한산코오롱하늘채아파트,우이천로 367,18/,43216.74,2024,816.1,4.48,0.0,0.0,0.0,0.0,0.0,4.48,NaN


In [121]:
building_underground_water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18206 entries, 0 to 18205
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   건축물명                 18201 non-null  object 
 1   건축물 위치               18008 non-null  object 
 2   층수(지상_지하)            15551 non-null  object 
 3   연면적                  3699 non-null   object 
 4   조사년도                 18206 non-null  int64  
 5   총발생량(톤)              17613 non-null  float64
 6   일평균발생량(톤/일)          18113 non-null  float64
 7   일평균이용현황(톤/일)_하천방류    1996 non-null   float64
 8   일평균이용현황(톤/일)_도로청소    1943 non-null   float64
 9   일평균이용현황(톤/일)_공원용수    3252 non-null   float64
 10  일평균이용현황(톤/일)_화장실세척   2788 non-null   float64
 11  일평균이용현황(톤/일)_건물용수    7800 non-null   float64
 12  미사용_하수도방류(톤/일)       17936 non-null  float64
 13  일평균이용현황(톤/일)_기타건물용수  396 non-null    object 
dtypes: float64(8), int64(1), object(5)
memory usage: 1.9+ MB


In [122]:
building_underground_water_df.isnull().sum()

건축물명                       5
건축물 위치                   198
층수(지상_지하)               2655
연면적                    14507
조사년도                       0
총발생량(톤)                  593
일평균발생량(톤/일)               93
일평균이용현황(톤/일)_하천방류      16210
일평균이용현황(톤/일)_도로청소      16263
일평균이용현황(톤/일)_공원용수      14954
일평균이용현황(톤/일)_화장실세척     15418
일평균이용현황(톤/일)_건물용수      10406
미사용_하수도방류(톤/일)           270
일평균이용현황(톤/일)_기타건물용수    17810
dtype: int64

In [123]:
# 일평균이용현황(톤/일)_하천방류, 도로청소, 공원용수, 화장실세척, 건물용수, 기타건물용수 컬럼은 결측치가 너무 많아서 사용하기 어려워 보임
building_underground_water_df.drop(columns=["일평균이용현황(톤/일)_하천방류", "일평균이용현황(톤/일)_도로청소", "일평균이용현황(톤/일)_공원용수", "일평균이용현황(톤/일)_화장실세척", "일평균이용현황(톤/일)_건물용수", "일평균이용현황(톤/일)_기타건물용수"], inplace=True)

In [124]:
# 연면적은 사용할 수도 있는데 결측치가 너무 많아서 제거 따로 구할 수 있는 방법도 없음
building_underground_water_df.drop(columns=["연면적"], inplace=True)

In [125]:
building_underground_water_df.isnull().sum()

건축물명                 5
건축물 위치             198
층수(지상_지하)         2655
조사년도                 0
총발생량(톤)            593
일평균발생량(톤/일)         93
미사용_하수도방류(톤/일)     270
dtype: int64

In [126]:
building_underground_water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18206 entries, 0 to 18205
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   건축물명            18201 non-null  object 
 1   건축물 위치          18008 non-null  object 
 2   층수(지상_지하)       15551 non-null  object 
 3   조사년도            18206 non-null  int64  
 4   총발생량(톤)         17613 non-null  float64
 5   일평균발생량(톤/일)     18113 non-null  float64
 6   미사용_하수도방류(톤/일)  17936 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 995.8+ KB


In [127]:
# "총발생량(톤)", "일평균발생량(톤/일)", "미사용_하수도방류(톤/일)" 이거 3개 모두가 결측치인 행은 지우기 (정보가 없음)
null_rows = building_underground_water_df[
    building_underground_water_df[["총발생량(톤)", "일평균발생량(톤/일)", "미사용_하수도방류(톤/일)"]].isnull().all(axis=1)
]

print(f"결측치만 있는 행 수: {len(null_rows)}")

building_underground_water_df = building_underground_water_df.drop(null_rows.index)

결측치만 있는 행 수: 7


In [128]:
# 건축물명, 건축물 위치가 결측치인 행 지우기
building_underground_water_df.dropna(subset=["건축물명", "건축물 위치"], inplace=True)

In [129]:
building_underground_water_df.isnull().sum()

건축물명                 0
건축물 위치               0
층수(지상_지하)         2576
조사년도                 0
총발생량(톤)            586
일평균발생량(톤/일)         86
미사용_하수도방류(톤/일)     263
dtype: int64

In [130]:
building_underground_water_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17996 entries, 0 to 18205
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   건축물명            17996 non-null  object 
 1   건축물 위치          17996 non-null  object 
 2   층수(지상_지하)       15420 non-null  object 
 3   조사년도            17996 non-null  int64  
 4   총발생량(톤)         17410 non-null  float64
 5   일평균발생량(톤/일)     17910 non-null  float64
 6   미사용_하수도방류(톤/일)  17733 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.1+ MB


In [131]:
building_underground_water_df.to_csv("data/processed/building_underground_water_processed.csv", index=False, encoding="utf-8-sig")

## 서울시 공사현장 유출지하수 공간정보(2019년)

In [132]:
construction_underground_water_df = pd.read_csv('data/raw_data/서울시 공사현장 유출지하수 공간정보(2019년).csv', encoding='cp949')

In [133]:
construction_underground_water_df.head()

,관리번호,자치구,등록년도,공사명,공사위치,X좌표,Y좌표
0,2019_0001,성동구,2019,뚝섬 부영복합빌딩 신축공사,서울시 성동구 성수동1가 685-701,203830.61,449268.719999
1,2019_0002,중랑구,2019,서울 양원지구 C-2BL 금강펜테리음 아파트 신축공사,서울시 중랑구 신내동,208774.00,456259.599999
2,2019_0003,광진구,2019,e편한세상 광진 그랜드파크 공동주택 신축공사,서울시 광진구 화양동,206306.80,449664.799999
3,2019_0004,강서구,2019,가양역 지식산업센터 신축공사,서울시 강서구 등촌동 628-9외 2필지,187230.41,451178.119999
4,2019_0005,중랑구,2019,서울 양원 힐데스하임 참좋은 아파트 신축공사,서울시 중랑구 망우동,209181.20,455109.999999


In [134]:
construction_underground_water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관리번호    12 non-null     object 
 1   자치구     12 non-null     object 
 2   등록년도    12 non-null     int64  
 3   공사명     12 non-null     object 
 4   공사위치    12 non-null     object 
 5   X좌표     12 non-null     float64
 6   Y좌표     12 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 804.0+ bytes


In [135]:
construction_underground_water_df.isnull().sum()

관리번호    0
자치구     0
등록년도    0
공사명     0
공사위치    0
X좌표     0
Y좌표     0
dtype: int64

In [136]:
# 의미 없는 컬럼 삭제
construction_underground_water_df.drop(columns=["관리번호"], inplace=True)

In [137]:
construction_underground_water_df.to_csv("data/processed/construction_underground_water_processed.csv", index=False, encoding="utf-8-sig")

## 서울시 도로굴착 공사 현황

In [158]:
road_construction_df = pd.read_csv('data/raw_data/서울시 도로굴착 공사 현황.csv', encoding='cp949')

In [159]:
road_construction_df.head()

,관리코드,구,동,공사명,착공일 ~ 준공일,신청자,처리상태,도로,포장,허가번호
0,인터넷-230306-01,성동구,행당동,동북선도시철도1공구 101정거장 특별피난계단<BR>(성동구 행당동 192-8 ~...,-- ~ --,인터넷 개인굴착 그룹,신청자접수완료,특별시도,차도/보도,NaN
1,인터넷-220604-01,종로구,이화동,이화동149번지 하수관연결공사<BR>(종로구 이화동 149 ~ 이화동 149),2022-06-15 ~ 2022-06-19,인터넷 개인굴착 그룹,착공계 접수,구도,차도/보도,종로구-2022-개인-00060
2,인터넷-220602-02,성북구,길음동,길음역세권 주택재개발사업중(지하연결통로공사)<BR>(성북구 길음동 875-1 ~...,-- ~ --,인터넷 개인굴착 그룹,허가,구도,차도,성북구-2022-개인-00067
3,인터넷-220602-02,성북구,길음동,길음역세권 주택재개발사업중(지하연결통로공사)<BR>(성북구 길음동 541-6 ~...,-- ~ --,인터넷 개인굴착 그룹,허가,특별시도,보도,성북구-2022-개인-00067
4,인터넷-211110-22,금천구,시흥동,신안산선 복선전철 3-1공구<BR>(금천구 시흥동 1001-11 ~ 시흥동 ...,2021-11-15 ~ 2021-11-30,인터넷 개인굴착 그룹,착공계 접수,특별시도,보도,금천구-2021-개인-00163


In [160]:
road_construction_df["처리상태"].unique()

array(['신청자접수완료', '착공계 접수', '허가', '협의완료', '협의요청', '검토완료', '허가증 교부',
       '징수확인'], dtype=object)

In [161]:
road_construction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   관리코드       533 non-null    object
 1   구          533 non-null    object
 2   동          533 non-null    object
 3   공사명        533 non-null    object
 4   착공일 ~ 준공일  533 non-null    object
 5   신청자        533 non-null    object
 6   처리상태       533 non-null    object
 7   도로         533 non-null    object
 8   포장         533 non-null    object
 9   허가번호       482 non-null    object
dtypes: object(10)
memory usage: 41.8+ KB


In [162]:
road_construction_df.isnull().sum()

관리코드          0
구             0
동             0
공사명           0
착공일 ~ 준공일     0
신청자           0
처리상태          0
도로            0
포장            0
허가번호         51
dtype: int64

In [163]:
# 필요없는 컬럼 제거 ("신청자" 컬럼도 지워야할지 고민중)
road_construction_df.drop(columns=["관리코드", "허가번호"], inplace=True)

In [164]:
road_construction_df["공사명"]

0      동북선도시철도1공구 101정거장 특별피난계단<BR>(성동구 행당동   192-8 ~...
1        이화동149번지 하수관연결공사<BR>(종로구 이화동   149 ~ 이화동   149)
2      길음역세권 주택재개발사업중(지하연결통로공사)<BR>(성북구 길음동   875-1 ~...
3      길음역세권 주택재개발사업중(지하연결통로공사)<BR>(성북구 길음동   541-6 ~...
4      신안산선 복선전철 3-1공구<BR>(금천구 시흥동   1001-11 ~ 시흥동   ...
                             ...                        
528    군자역 8번 출입구 승강편의시설 설치공사(에스컬레이터)<BR>(광진구 능동   천호...
529    노원 외 1역 승강편의시설 설치공사(노원역)<BR>(노원구 상계동   동일로 140...
530    까치산 외 1역 승강편의시설 설치공사(39공구, 까치산역)_원인자<BR>(강서구 화...
531    교대역 13출구 엘리베이터 설치공사<BR>(서초구 서초동   서초중앙로 116일원 ...
532    교대역 13출구 엘리베이터 설치공사<BR>(서초구 서초동   서초중앙로 116일원 ...
Name: 공사명, Length: 533, dtype: object

In [165]:
# 공사명에서 공사명과 주소 분리
split_cols = road_construction_df["공사명"].str.split("<BR>", n=1, expand=True)

road_construction_df["공사명"] = split_cols[0].str.strip()
road_construction_df["주소"] = split_cols[1].str.strip("() ").replace("", pd.NA)

In [166]:
road_construction_df.head()

,구,동,공사명,착공일 ~ 준공일,신청자,처리상태,도로,포장,주소
0,성동구,행당동,동북선도시철도1공구 101정거장 특별피난계단,-- ~ --,인터넷 개인굴착 그룹,신청자접수완료,특별시도,차도/보도,성동구 행당동 192-8 ~ 행당동 192-40
1,종로구,이화동,이화동149번지 하수관연결공사,2022-06-15 ~ 2022-06-19,인터넷 개인굴착 그룹,착공계 접수,구도,차도/보도,종로구 이화동 149 ~ 이화동 149
2,성북구,길음동,길음역세권 주택재개발사업중(지하연결통로공사),-- ~ --,인터넷 개인굴착 그룹,허가,구도,차도,성북구 길음동 875-1 ~ 길음동 875-1
3,성북구,길음동,길음역세권 주택재개발사업중(지하연결통로공사),-- ~ --,인터넷 개인굴착 그룹,허가,특별시도,보도,성북구 길음동 541-6 ~ 길음동 541-154
4,금천구,시흥동,신안산선 복선전철 3-1공구,2021-11-15 ~ 2021-11-30,인터넷 개인굴착 그룹,착공계 접수,특별시도,보도,금천구 시흥동 1001-11 ~ 시흥동 1001-11


In [167]:
# 착공일 ~ 준공일 컬럼에서 -- ~ -- 값 null로 하고 분리, date형식으로 바꾸기
import numpy as np
road_construction_df["착공일 ~ 준공일"] = road_construction_df["착공일 ~ 준공일"].replace("-- ~ --", np.nan)

In [168]:
road_construction_df[["착공일", "준공일"]] = road_construction_df["착공일 ~ 준공일"].str.split("~", expand=True)

road_construction_df["착공일"] = pd.to_datetime(road_construction_df["착공일"].str.strip(), errors="coerce")
road_construction_df["준공일"] = pd.to_datetime(road_construction_df["준공일"].str.strip(), errors="coerce")

In [169]:
road_construction_df.drop(columns=["착공일 ~ 준공일"], inplace=True)

In [170]:
road_construction_df["공사일수"] = (
    road_construction_df["준공일"] - road_construction_df["착공일"]
).dt.days

In [171]:
road_construction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   구       533 non-null    object        
 1   동       533 non-null    object        
 2   공사명     533 non-null    object        
 3   신청자     533 non-null    object        
 4   처리상태    533 non-null    object        
 5   도로      533 non-null    object        
 6   포장      533 non-null    object        
 7   주소      533 non-null    object        
 8   착공일     372 non-null    datetime64[ns]
 9   준공일     372 non-null    datetime64[ns]
 10  공사일수    372 non-null    float64       
dtypes: datetime64[ns](2), float64(1), object(8)
memory usage: 45.9+ KB


In [172]:
road_construction_df.head()

,구,동,공사명,신청자,처리상태,도로,포장,주소,착공일,준공일,공사일수
0,성동구,행당동,동북선도시철도1공구 101정거장 특별피난계단,인터넷 개인굴착 그룹,신청자접수완료,특별시도,차도/보도,성동구 행당동 192-8 ~ 행당동 192-40,NaT,NaT,NaN
1,종로구,이화동,이화동149번지 하수관연결공사,인터넷 개인굴착 그룹,착공계 접수,구도,차도/보도,종로구 이화동 149 ~ 이화동 149,2022-06-15,2022-06-19,4.0
2,성북구,길음동,길음역세권 주택재개발사업중(지하연결통로공사),인터넷 개인굴착 그룹,허가,구도,차도,성북구 길음동 875-1 ~ 길음동 875-1,NaT,NaT,NaN
3,성북구,길음동,길음역세권 주택재개발사업중(지하연결통로공사),인터넷 개인굴착 그룹,허가,특별시도,보도,성북구 길음동 541-6 ~ 길음동 541-154,NaT,NaT,NaN
4,금천구,시흥동,신안산선 복선전철 3-1공구,인터넷 개인굴착 그룹,착공계 접수,특별시도,보도,금천구 시흥동 1001-11 ~ 시흥동 1001-11,2021-11-15,2021-11-30,15.0


In [173]:
road_construction_df.to_csv("data/processed/road_construction_processed.csv", index=False, encoding="utf-8-sig")

## 지하철역 깊이 데이터

In [83]:
subway_depth = pd.read_csv('data/raw_data/서울교통공사_역사심도정보_20241104.csv', encoding='cp949')
subway_coordinate = pd.read_csv('data/raw_data/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20241031.csv', encoding='cp949')

In [84]:
subway_depth.head()

,연번,호선,역명,층수,형식,지반고,레일면고,선로기준정거장깊이,정거장깊이,비고
0,1,1,서울,B2,섬식,129.99,117.04,12.95,11.85,"4호선,경의중앙선,공항철도환승"
1,2,1,시청,B2,상대식,129.97,118.82,11.15,10.05,2호선환승
2,3,1,종각,B2,상대식,128.77,116.24,12.53,11.43,NaN
3,4,1,종로3가,B2,상대식,124.38,112.04,12.34,11.24,"3,5호선환승"
4,5,1,종로5가,B2,상대식,121.75,109.26,12.49,11.39,NaN


In [85]:
subway_coordinate.head()

,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자
0,1,1,150,서울,37.553150,126.972533,1974-02-28
1,2,1,151,시청,37.563590,126.975407,1974-08-15
2,3,1,152,종각,37.570203,126.983116,1974-08-15
3,4,1,153,종로3가,37.570429,126.992095,1974-08-15
4,5,1,154,종로5가,37.570971,127.001900,1974-03-31


In [86]:
for df in [subway_depth, subway_coordinate]:
    df['역명'] = df['역명'].astype(str).str.strip().str.replace(" ", "")
    df['호선'] = df['호선'].astype(str).str.strip().str.replace(" ", "")

In [87]:
# 조인키로 쓸 set 만들기
depth_keys = set(zip(subway_depth["호선"], subway_depth["역명"]))
coord_keys = set(zip(subway_coordinate["호선"], subway_coordinate["역명"]))

# depth에는 있지만 coordinate에는 없는
only_in_depth = depth_keys - coord_keys

# coordinate에는 있지만 depth에는 없는
only_in_coord = coord_keys - depth_keys

# depth에는 있지만 coordinate에는 없는 행
not_in_coord = subway_depth[subway_depth.apply(
    lambda row: (row['호선'], row['역명']) in only_in_depth, axis=1)]

# coordinate에는 있지만 depth에는 없는 행
not_in_depth = subway_coordinate[subway_coordinate.apply(
    lambda row: (row['호선'], row['역명']) in only_in_coord, axis=1)]

In [88]:
print("depth에는 있는데 좌표에 없는 역:")
print(not_in_coord[['호선', '역명']])

depth에는 있는데 좌표에 없는 역:
    호선      역명
12   2    을지3가
13   2    을지4가
41   2   구로디지털
102  4   미아삼거리
112  4     서울역
141  5     애오게
147  5  동대문운동장
165  5      둔촌
185  6     DMC
243  7      이수
245  7     숭실대
248  7     신대방
255  7      광명
276  8  암사역사공원


In [89]:
print("\n좌표에는 있는데 depth에 없는 역:")
print(not_in_depth[['호선', '역명']])


좌표에는 있는데 depth에 없는 역:
    호선         역명
11   2      을지로3가
12   2      을지로4가
40   2    구로디지털단지
102  4      미아사거리
112  4         서울
141  5        애오개
147  5  동대문역사문화공원
165  5        둔촌동
185  6   디지털미디어시티
243  7      총신대입구
245  7      숭실대입구
248  7     신대방삼거리
255  7      광명사거리


In [90]:
subway_depth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   연번         277 non-null    int64  
 1   호선         277 non-null    object 
 2   역명         277 non-null    object 
 3   층수         277 non-null    object 
 4   형식         277 non-null    object 
 5   지반고        277 non-null    object 
 6   레일면고       277 non-null    object 
 7   선로기준정거장깊이  277 non-null    float64
 8   정거장깊이      277 non-null    float64
 9   비고         109 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 21.8+ KB


In [91]:
subway_coordinate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            276 non-null    int64  
 1   호선            276 non-null    object 
 2   고유역번호(외부역코드)  276 non-null    int64  
 3   역명            276 non-null    object 
 4   위도            276 non-null    float64
 5   경도            276 non-null    float64
 6   작성일자          276 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 15.2+ KB


In [92]:
# 직접바꾸기
subway_depth.loc[subway_depth['역명'] == '을지3가', '역명'] = '을지로3가'
subway_depth.loc[subway_depth['역명'] == '을지4가', '역명'] = '을지로4가'
subway_depth.loc[subway_depth['역명'] == '삼성', '역명'] = '삼성' # ?
subway_depth.loc[subway_depth['역명'] == '구로디지털', '역명'] = '구로디지털단지'
subway_depth.loc[subway_depth['역명'] == '미아삼거리', '역명'] = '미아사거리'
subway_depth.loc[subway_depth['역명'] == '서울역', '역명'] = '서울'
subway_depth.loc[subway_depth['역명'] == '애오게', '역명'] = '애오개'
subway_depth.loc[subway_depth['역명'] == '동대문운동장', '역명'] = '동대문역사문화공원'
subway_depth.loc[subway_depth['역명'] == '둔촌', '역명'] = '둔촌동'
subway_depth.loc[subway_depth['역명'] == 'DMC', '역명'] = '디지털미디어시티'
subway_coordinate.loc[subway_coordinate['역명'] == '총신대입구', '역명'] = '이수'
subway_depth.loc[subway_depth['역명'] == '총신대입구', '역명'] = '이수'
subway_depth.loc[subway_depth['역명'] == '숭실대', '역명'] = '숭실대입구'
subway_depth.loc[(subway_depth['역명'] == '신대방') & (subway_depth['호선'] == '7'), '역명'] = '신대방삼거리'
subway_depth.loc[subway_depth['역명'] == '삼성', '역명'] = '삼성'
subway_depth.loc[subway_depth['역명'] == '광명', '역명'] = '광명사거리'

In [93]:
# 깊이정보 데이터에 위경도 좌표 추가하기
subway_depth.columns = subway_depth.columns.str.strip()
subway_coordinate.columns = subway_coordinate.columns.str.strip()

merged = pd.merge(
    subway_depth,
    subway_coordinate[["호선", "역명", "위도", "경도"]],
    how="left",
    on=["호선", "역명"]
)

merged.head()

,연번,호선,역명,층수,형식,지반고,레일면고,선로기준정거장깊이,정거장깊이,비고,위도,경도
0,1,1,서울,B2,섬식,129.99,117.04,12.95,11.85,"4호선,경의중앙선,공항철도환승",37.553150,126.972533
1,2,1,시청,B2,상대식,129.97,118.82,11.15,10.05,2호선환승,37.563590,126.975407
2,3,1,종각,B2,상대식,128.77,116.24,12.53,11.43,NaN,37.570203,126.983116
3,4,1,종로3가,B2,상대식,124.38,112.04,12.34,11.24,"3,5호선환승",37.570429,126.992095
4,5,1,종로5가,B2,상대식,121.75,109.26,12.49,11.39,NaN,37.570971,127.001900


In [94]:
merged.isnull().sum()

연번             0
호선             0
역명             0
층수             0
형식             0
지반고            0
레일면고           0
선로기준정거장깊이      0
정거장깊이          0
비고           168
위도             1
경도             1
dtype: int64

In [95]:
missing_coords = merged[merged['위도'].isnull() & merged['경도'].isnull()]
print(missing_coords[['호선', '역명', '위도', '경도']])

    호선      역명  위도  경도
276  8  암사역사공원 NaN NaN


In [96]:
merged = merged[merged['역명'] != '암사역사공원']

In [97]:
merged.to_csv("data/processed/subway_depth.csv", index=False, encoding="utf-8-sig")